In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import math
from scipy import stats
import scanpy as sc

In [2]:
#Import dataset1 for either training or testing
global_dir_path = "C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data"
#4wks
# dir_path = global_dir_path + '/2023_0809_0810_Breath_Eml4Alk_4wks_multiplex/Spectra/Peaks'
#5wks
# dir_path = global_dir_path + '/2023_0816_0817_Breath_Eml4Alk_5wks_multiplex/All samples_spectra/Peaks'

#6wks
# dir_path = global_dir_path + '/2023_May_June_Eml4Alk_6to8wks_spectra/2023_05_19_Breath_Eml4Alk_6wks_calibrated_spectra/Peaks'

#6wks August
# dir_path = global_dir_path + '/2023_08_24_Breath_EmlAlk_6wks_multiplex/Spectra/Peaks'

#4wks October
# dir_path = global_dir_path + '/2023_10_19_Breath_Eml4Alk_4wk_5plex/Spectra/Peaks'

#5wks October
# dir_path = global_dir_path + '/2023_10_26_Breath_Eml4Alk_5wks_cohort2/spectra/Peaks'

#6wks October failed longitudinal - not used
# dir_path = global_dir_path + '/2023_10_23_Breath_Eml4-Alk_6wks_cohort1/spectra/Peaks'

#6wks Jan2024
# dir_path = global_dir_path + '/2024_0116_0117_Breath_Eml4aLK_6w_endo/Peaks'

#5wks Feb2024
# dir_path = global_dir_path + '/2024_02_09_Eml4-Alk_5wks_multiplex/Spectra/Peaks'

#6wks Feb2024
dir_path = global_dir_path + '/2024_02_16_Eml4Alk_6wks_multiplex/spectra/Peaks'


dir_list = sorted(os.listdir(dir_path))
print(dir_list)

# dir_list2 = sorted(os.listdir(dir_path2))
# print(dir_list2)

['2024_02_16_W6_EA4_1_peaks.xlsx', '2024_02_16_W6_EA4_2_peaks.xlsx', '2024_02_16_W6_EA4_3_peaks.xlsx', '2024_02_16_W6_EA4_4_peaks.xlsx', '2024_02_16_W6_EA5_1_peaks.xlsx', '2024_02_16_W6_EA5_2_peaks.xlsx', '2024_02_16_W6_EA5_3_peaks.xlsx', '2024_02_16_W6_EA5_4_peaks.xlsx', '2024_02_16_W6_EA5_5_peaks.xlsx', '2024_02_16_W6_Healthy2_1_peaks.xlsx', '2024_02_16_W6_Healthy2_2_peaks.xlsx', '2024_02_16_W6_Healthy2_3_peaks.xlsx', '2024_02_16_W6_Healthy2_4_peaks.xlsx', '2024_02_16_W6_Healthy2_5_peaks.xlsx', 'Int_ratio_r1']


In [3]:
#find and remove exogenous signals
def remove_exogenous(file, peaks):
    intensity = []
    index = []
    mz_list = []
    for peak in peaks:
        for i in range(len(file)):
            mz = file.iloc[i, file.columns.get_loc('m/z')]
            #remove if the integer of the m/z is the same as the exogenous molecules
            if int(mz) == peak: 
                intensity.append(file.iloc[i, file.columns.get_loc('Intensity (corr)')]) #change to intensity
                mz_list.append(df.iloc[i, df.columns.get_loc('m/z')])
                index.append(i)
    
    file = file.drop(index)
    #the original dataframe is not modified unless setting inplace=True
    return file

#calibrate peak ratios to the water peak
def get_water_intensity(file):
    index = []
    intensity_water = []
    for i in range(len(file)):
        mz = file.iloc[i, file.columns.get_loc('m/z')]
        if round(mz,1) < 19.5 and round(mz,1) > 18.0: #change from 18.5 to 18.0
            intensity_water.append(file.iloc[i, file.columns.get_loc('Intensity (corr)')]) #change to intensity
            index.append(i)
    
    max_val = max(intensity_water)
    idx = index[intensity_water.index(max_val)]
    
    return file.iloc[idx, file.columns.get_loc('m/z')], file.iloc[idx, file.columns.get_loc('Intensity (corr)')] #change

def get_intensity_ratio(file): #change
    result = np.zeros(len(file))
    for i in range(len(file)):
        index = file.iloc[i, file.columns.get_loc('Intensity (corr)')] #change
        result[i] = index/get_water_intensity(file)[1] #change
    return result

In [4]:
def clean_mass(file, file_idx):
    mz_ls = []
    intr_ls = []
    int_ls = []

    i = 0
    while i < len(file)-1:

        local_mz = []
        local_intr = []
        local_int = []

        for x in range(1, len(file)-1 ,1):
            mz = file.iloc[i, file.columns.get_loc('m/z')]
            int_ratio = file.iloc[i, file.columns.get_loc('Intensity_ratio')]
            int_corr = file.iloc[i, file.columns.get_loc('Intensity (corr)')]

            if i+x < len(file): 
                if mz not in local_mz:
                    local_mz.append(mz)
                    local_intr.append(int_ratio)
                    local_int.append(int_corr)

                mzx = file.iloc[i+x, file.columns.get_loc('m/z')]
                int_ratiox = file.iloc[i+x, file.columns.get_loc('Intensity_ratio')]
                int_corrx = file.iloc[i+x, file.columns.get_loc('Intensity (corr)')]

                if round(mzx, 1) == round(mz, 1): #change, look at three digits after the decimal point and round to 2

                    local_mz.append(mzx)
                    local_intr.append(int_ratiox)
                    local_int.append(int_corrx)

                else:
                    stopper = x
                    break
            else:
                break

        #find local max and append to the final list         
        max_intr = max(local_intr)
        max_index = local_intr.index(max_intr)

        mz_ls.append(round(local_mz[max_index],1)) #change
        intr_ls.append(local_intr[max_index])
        int_ls.append(local_int[max_index])

        i = i + stopper


    zip_result = zip(mz_ls, intr_ls, int_ls)
    columns=['m/z', 'Intensity_ratio', 'Intensity']
    
    df1 = pd.DataFrame(zip_result, columns=columns)

    #Hard code
    #Remove all endogenous peaks, water, ion source, and calibration peaks
    peaks = [14, 15, 17, 18, 19, 20, 21, 27, 28, 29, 30, 36, 37, 39, 51, 52, 53, 67, 68, 77, 78, 149, 150, 249, 250, 
              203, 204, 330, 331] 
    #remove 27 (HCN), 28 (CO+, N2+), 29 (N2H+), 30 (NO+), 36 (two water), 37 (two water)

    index = []
    for peak in peaks:
        for i in range(len(df1)):
            mz = df1.iloc[i, df1.columns.get_loc('m/z')]
            #remove if the integer of the m/z is the same as the exogenous molecules
            if int(mz) == peak: 
                index.append(i)

    df1= df1.drop(index)

    filename = dir_path + '/Int_ratio_r1/' + dir_list[file_idx] #change, add a new file
    df1.to_excel(filename)

In [5]:
for i in range(len(dir_list)):
    if ".xlsx" in dir_list[i]:
        name = dir_path + '/' + dir_list[i]
        with open(name, mode="r", encoding="iso-8859-1") as file:
            df = pd.read_excel(name, index_col=0)

        #df = remove_exogenous(df, peaks)
        ratio = get_intensity_ratio(df)
        df['Intensity_ratio'] = ratio

        clean_mass(df, i) 

In [6]:
# peaks = [27,28,29,30,51,52,53,67,68,77,78,149,150,249,250,203,204,330,331] 